In [1]:
from init import *
from hytraj import HyData, HyGen

autotime loaded.
black loaded.
black loaded.
time: 871 ms


<IPython.core.display.Javascript object>

In [2]:
working = "/home/pankaj/Desktop/hysplit/"
metdir = "/home/pankaj/Desktop/met/"
outdir = "/home/pankaj/Desktop/hysplit/out/"
hysplit = "/home/pankaj/Desktop/hysplit/hyts_std"

time: 440 µs


<IPython.core.display.Javascript object>

In [3]:
height = 500
year = 2010
run_time = -360
dates = pd.date_range("2010-02-01", freq="24H", end="2010-02-15")
dates

DatetimeIndex(['2010-02-01', '2010-02-02', '2010-02-03', '2010-02-04',
               '2010-02-05', '2010-02-06', '2010-02-07', '2010-02-08',
               '2010-02-09', '2010-02-10', '2010-02-11', '2010-02-12',
               '2010-02-13', '2010-02-14', '2010-02-15'],
              dtype='datetime64[ns]', freq='24H')

time: 3.37 ms


<IPython.core.display.Javascript object>

In [4]:
stations = {
    "Davis": (-69, 78),
    "South Pole": (-90, 335),
    "Neumayer": (-71, 352),
}

time: 466 µs


<IPython.core.display.Javascript object>

In [5]:
hy = HyGen(
    year,
    stations,
    height,
    run_time,
    working,
    metdir,
    outdir,
    met_type="ncep",
    exe=hysplit,
)
hy = hy.run(dates, vertical=0, model_top=10000.0)

100%|██████████| 15/15 [20:45<00:00, 83.02s/it] 

time: 20min 45s


<IPython.core.display.Javascript object>

In [6]:
list(stations.keys())

['Davis', 'South Pole', 'Neumayer']

time: 1.47 ms


<IPython.core.display.Javascript object>

In [10]:
files = sorted(glob.glob("/home/pankaj/Desktop/hysplit/out/ncep*"))
hy = HyData(files, list(stations.keys()))
data = hy.read()
data

<xarray.Dataset>
Dimensions:     (geo: 4, step: 361, time: 15)
Coordinates:
  * step        (step) int64 0 1 2 3 4 5 6 7 ... 353 354 355 356 357 358 359 360
  * time        (time) datetime64[ns] 2010-02-01 2010-02-02 ... 2010-02-15
  * geo         (geo) <U3 'lat' 'lon' 'alt' 'pre'
Data variables:
    Davis       (geo, step, time) float64 -69.0 -69.0 -69.0 ... 742.3 540.7
    South Pole  (geo, step, time) float64 -90.0 -90.0 -90.0 ... 643.9 830.0
    Neumayer    (geo, step, time) float64 -71.0 -71.0 -71.0 ... 407.4 636.4

time: 161 ms


<IPython.core.display.Javascript object>